# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

url = "http://api.openweathermap.org/data/2.5/weather?q="
appid = "&appid="
city_df = []

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

624

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# loop thru (cities) and print each name/id
counter = 1

country= []
cityn = []
lats = []
lngs = []
temps = []
max_temps = []
humidities = []
clouds = []
winds = []

for cityname in cities:
    if counter < 500:
        try:
            query_url = url + cityname + appid + weather_api_key
            city_request = requests.get(query_url).json()
            city_name = city_request['name']
            city_lat = city_request['coord']['lat']
            city_long = city_request['coord']['lon']
            city_temp = city_request['main']['temp']
            city_max = city_request['main']['temp_max']
            city_hum = city_request['main']['humidity']
            city_cloud = city_request['clouds']['all']
            city_wind = city_request['wind']['speed']
            # get country info
            country_info = city_request['sys']['country']
            # append city name
            cityn.append(city_name)
            # append country info
            country.append(country_info)
            lats.append(city_lat)
            lngs.append(city_long)
            temps.append(city_temp)
            max_temps.append(city_max)
            humidities.append(city_hum)
            clouds.append(city_cloud)
            winds.append(city_wind)
            print("city name is " + str(city_name) + " and the position is: " + str(counter))
            counter = counter + 1
        except:
            break

city name is Pangnirtung and the position is: 1


In [4]:
lats

[66.1451]

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# api call template: api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}
#sample call for city yellowknife

#.append not adding
#no .append results in single float point

city_df = []

        

city_df = {
    'country' : country,
           'cityn' : cityn,
    'lats' : lats,
    'lngs' : lngs,
    "temps" : temps,
    'max_temps' : max_temps,
    'humidities' : humidities,
    'clouds' : clouds,
    'winds' : winds
               
} 
city_df





{'country': ['CA'],
 'cityn': ['Pangnirtung'],
 'lats': [66.1451],
 'lngs': [-65.7125],
 'temps': [245.23],
 'max_temps': [245.23],
 'humidities': [81],
 'clouds': [97],
 'winds': [0.61]}

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

AttributeError: 'dict' object has no attribute 'head'

In [6]:
#convet to df
cities_df = pd.DataFrame(city_df)

In [10]:
#  Get the indices of cities that have humidity over 100%.


,country,cityn,lats,lngs,temps,max_temps,humidities,clouds,winds
0,CA,Pangnirtung,66.1451,-65.7125,245.23,245.23,81,97,0.61


In [8]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [26]:


#export to csv 
cities_df.to_csv("output_data/cities.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [27]:
cities_df["latn"] = np.where(cities_df.OUTCOME > 0, cities_df.OUTCOME, 0)
cities_df["lats"] = np.where(cities_df.OUTCOME < 0, - cities_df.OUTCOME, 0) 

AxesSubplot:xlabel='Latitude', ylabel='Temperature'
cities_df.plot.scatter(x='lat', y='temp' )



SyntaxError: invalid syntax (<ipython-input-27-57dc32b51209>, line 4)

## Latitude vs. Humidity Plot

In [17]:
cities_df.plot.scatter(x='lat', y='humidity' )
AxesSubplot:xlabel='Latitude', ylabel='Humidity'
    

SyntaxError: invalid syntax (<ipython-input-17-8b17a16650e0>, line 2)

## Latitude vs. Cloudiness Plot

In [ ]:
cities_df.plot.scatter(x='lat', y='all' )
AxesSubplot:xlabel='Latitude', ylabel='Cloudiness'

## Latitude vs. Wind Speed Plot

In [ ]:
cities_df.plot.scatter(x='lat', y='speed' )
AxesSubplot:xlabel='Latitude', ylabel='Wind Speed'

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['temp_max']
y_value = cities_df['latn'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Max Temp")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['temp_max']
y_value = cities_df['lats'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Max Temp")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['humidity']
y_value = cities_df['latn'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Humidity")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['humidity']
y_value = cities_df['lats'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Humidity")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['all']
y_value = cities_df['latn'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Cloudiness")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['all']
y_value = cities_df['lats'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Cloudiness")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['speed']
y_value = cities_df['latn'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Wind Speed")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_value = cities_df['speed']
y_value = cities_df['lats'] 
                       
(slope, intercept, rvalue, pvalue, std)= linregress(x_value, y_value)
regress_value =  x_value * slope + intercept   
line_eq = "y= " + str(slope) + "x " + str(intercept)
plt.scatter(x_value, y_value)
plt.xlabel("Wind Speed")
plt.ylabel("Latitude")
plt.plot(x_value, regress_value)
print("the correlation coefficient is " + str(rvalue))
#print("the r squared value is " + str(rvalue**2))
plt.show()